# Underactuated Systems Tutorial

### Context and Definitions
Simply, an underactuated system means that the number of independent control inputs is fewer than the number of degrees of freedom a system is trying to control.

First, lets talk about some math and establish our formula and symbols. The technical definition of an underactuated system involves some math, but we can break it down.

1. Dynamics of mechanical systems are second order because they follow F=ma. 

2. This means that we can represent the state of the system with a vector of positions and vector of velocities. For example, $q$ and $\dot{q}$ and $\ddot{q}$

3. First lets represent $\ddot{q}$ as a function of $q,\ \dot{q},\ u(control\ vector),t(time)$. 

     $$\ddot{q}=f(q,\dot{q},u,t)$$
    
4. We can then split up the accelerations by controlled and uncontrolled.
    
     $$\ddot{q}=f(q,\dot{q},t) + u*f(q,\dot{q},t)$$
    
5. A system is considered fully actuated if the rank of controlled accelerations (aka the f multiplied by u) is equal to the dimensions of $q$.
6. A system is considered underactuated if the rank of the controlled forces is less than the dimensions of $q$.

### Linearization of the System
Now we want to linearize the system. The reason for linearization is that LQR (a type of cost function that you will use later) only works for linear systems. Thus, we want to approximate the nonlinear system defined above by linearizing and then we can use LQR. In order to linearize the system about a fixed point, $(X^*, U^*)$, we use a Taylor expansion of the form:
$$\dot{X} = f(X, U) \approx f(X^*,U^*) + \frac{\partial{f}}{\partial{X}}_{X=X^*,U=U^*}(X−X^*) + \frac{\partial{f}}{\partial{U}}_{X=X^*,U=U^*}(U−U^*)$$
To simplify things, instead of having $\frac{\partial{f}}{\partial{X}}_{X=X^*,U=U^*}$, we want to use a matrix, $A$ to represent the abstract form of $\frac{\partial{f}}{\partial{X}}$. Then we can plug in a point $(X^*, U^*)$ to evaluate $A$ at a particular point.

Write $A = \frac{\partial{f}}{\partial{X}}$ as a list of lists $([[]])$ so it can be viewed as a matrix. For example, the Identity Matrix:
<table>
    <tr>
        <th>1</th>
        <th>0</th>
        <th>0</th>
    </tr>
    <tr>
        <th>0</th>
        <th>1</th>
        <th>0</th>
    </tr>
    <tr>
        <th>0</th>
        <th>0</th>
        <th>1</th>
    </tr>
</table>
would look like $[[1, 0, 0], [0, 1, 0], [0, 0, 1]]$ as a list of lists in Python.

# LQR-RRT\*

### What is Linear Quadratic Regulator (LQR)?
Linear Quadratic Regulator (LQR) is a control method that tries to control a system with linear dynamics and quadratic cost function. LQR finds a linear control policy (control gain) that optimizes the given quadratic cost function. Linear system dynamics are given as the following linear differential equation. Note, A and B are matrices of proper sizes.

$$\dot{x} = Ax + Bu + w$$

Quadratic cost function is given as the following integral form. Here, we consider infinite horizon LQR for convenience, meaning that cost function is defined over $t=0$ to $t=\infty$. Q, R, and N are also matrices of proper sizes.

$$J = \int_{0}^{\infty} (x^TQx + u^TRu + 2x^TNu) dt$$

Let’s assume a linear feedback controller of the following form. K is a matrix of proper size.

$$u=-Kx$$

Then, optimal gain is given by the following form. P is also a matrix of proper size.

$$K=R^{-1}(B^TP+N^T)$$

P can be obtained from the following algebraic Riccati equation.

$$A^TP+PA-(PB+N)R^{-1}(B^TP+N^T)+Q=0$$

Since how optimal gain for LQR is obtained is out of scope for this problem set, we will just use the analytical form given above for obtaining optimal gain.

### RRT
Now, let’s go over briefly about what RRT and RRT\* are.

Rapidly-exploring Random Tree (RRT) is a sampling based motion planning algorithm. The main motivation of RRT’s invention was to deal with underactuated dynamics in motion planning problems. You can see the result of using RRT to perform motion planning for a underactuated robot in the following figure.

<img src="photos/RRT.png" height="500" width="500">

We provide you the pseudo code for RRT.

```python
def RRT(map_1, x_initial, n_tree, max_dist):
    
    tree = initiallize_tree(x_initial)
    
    for i in range (0, n_tree):
        x_rand = Sample(map_1)
        x_nearest = Nearest(tree, x_rand)
        [x_new, traj_new] = Steer(x_nearest, x_rand)
        tree = add_to_tree(x_new, traj_new)
        
    return tree
```

The input to RRT algorithm is the configuration of initial state. The algorithm outputs the tree that starts from the initial configuration and spans out over the given configuration space. RRT algorithm iterates for number of total nodes of trees which is specified by users. Starting from the initial tree with only initial configuration, RRT incrementally expands the tree in every iteration. In each iteration, a new configuration is sampled randomly. Then, RRT finds a node (configuration) in the existing tree that is nearest to the newly sampled configuration. Here, the metric for determining what the nearest node depends on the application. In many cases, Euclidean distance is used for convenience. After nearest configuration is selected, RRT tries to connect to the newly sampled node from the nearest configuration. Here, connection doesn’t have to meet the newly sampled node exactly since the new node is going to be connected to the tree as leaf of the tree. This is why RRT is suitable for handling underactuated dynamical systems (for underactuated dynamical systems, finding trajectory that connects given two points exactly, called Two Point Boundary Value Problem (TPBVP), is a nontrivial problem). After expansion, we add the edge and the resulting end configuration from the connection to the existing tree and start a new iteration.

### RRT\*
RRT\* is the optimal version of RRT. In each iteration, it locally fixes RRT so that only the optimal trajectories are kept in the tree given the locally fixing diameter. This local pruning of the tree ensures optimality of resulting trajectories between initial configuration and any node within the tree. You can check the optimality of RRT\* in the following figure.

<img src="photos/RRT_star.png" height="500" width="500">

Only additional step required for RRT\*, compared to RRT, is the rewiring step, which does the local pruning of the tree. Once the density of the sampled configurations in the configuration space is large enough, the resulting tree finds optimal trajectory from initial configuration to any sampled configurations. However, the rewiring step requires solving a TPBVP, meaning that the connection between given two nodes must be exact following the given system dynamics. For this reason, applying RRT\* to underactuated system efficiently is still an open research topic. We provide you the pseudo code for RRT\*.

```python
def RRT_star(map_1, x_initial, n_tree, max_dist):
    
    tree = initiallize_tree(x_initial)
    
    for i in range (0, n_tree):
        x_rand = Sample(map_1)
        x_nearest = Nearest(tree, x_rand)
        [x_new, traj_new] = Steer(x_nearest, x_rand)
        x_near_set = Near(tree, x_new)
        [x_min, traj_min] = Choose_parent(x_near_set, x_new)
        
        if Collision_free(traj_min):
            tree = add_to_tree(x_new, traj_min)
            tree = Rewire(tree, x_near_set, x_new)
            
    return tree

def Rewire(tree, x_near_set, x_new):
    for x_near in x_near_set:
        [x, traj] = Steer(x_new, x_near)
        if Cost(x_new) + Cost(traj) < Cost(x_near):
            if Collision_free(traj):
                x_parent = Parent(x_near)
                tree = Remove_traj(x_parent, x_near)
                tree = add_traj(x_new, x_near)
    
    return tree
```

#### Choose Parent Function
Within RRT\*, there are two important subroutines; *Choose Parent* subroutine and *Rewire* subroutine. You are going to implement *Choose Parent* subroutine. In *Choose Parent* subroutine, instead of directly connecting the sampled node to the nearest node as in RRT, sampled node is connected to the node in the tree that yields the lowest cost starting from the initial configuration. Below is some pseudocode for *Choose Parent* subroutine.

```python
def Choose_parent(x_near_set, x_new):
    
    min_cost = inf
    x_min = nil
    traj_min = nil
    
    for x_near in x_near_set:
        [x, traj] = Steer(x_near, x_new)
        if Cost(x_near) + Cost(traj) < min_cost:
            min_cost = Cost(x_near) + Cost(traj)
            x_min = x_near
            traj_min = traj
    
    return x_min, traj_min
```

Let's go over *Choose Parent* subroutine briefly. First, we initialize minimum cost variable from initial configuration to sampled node with infinity. Node in tree that connects to the sampled node and the trajectory between the two nodes are initialized with *nil* as well. For each node in near nodes set, we steer to the sampled node. If the cost to the sampled node via the node in near nodes set is smaller than one from the previous iteration, we change the minimum cost variable to the new minimum cost. We also change minimum cost node and trajectory to the corresponding ones as well.

### LQR-RRT\*
LQR-RRT\* avoids solving the TPBVP exactly by using the concept of LQR control method. It finds the set of LQR control policies that guides the initial configuration to any other configurations in the configuration space. If the given dynamical system can be stably steered to an objective configuration using the LQR control policy (this can be proved using Lyapunov stability analysis for a given dynamics), this means we can steer the dynamical system to a vicinity of the objective configuration (This was checked in the above code). If the system is linear, we only need one LQR control policy centered around the objective configuration. If, however, the system dynamics is nonlinear, we might need a sequence of LQR control policies centered around a sequence of configuration nodes to steer the system to the objective configuration. If the tree is dense enough and the system can be locally stabilized around each linearized points in the configuration with LQR control policy, then we can make the system converge to the objective configuration in the end by following a sequence of LQR control policy. You can think of this as following a sequence of funnels to get to the objective configuration.

<img src="photos/LQR_RRT_star.png" height="200" width="200">

Another point of LQR-RRT\* is that instead of using Euclidean distance for the cost to determine the nearest node and a set of near nodes from a randomly sampled configuration, it uses the cost calculated from LQR controller. This makes the tree expansion a lot more efficient since LQR controller based cost is a lot more accurate than Euclidean distance for underactuated systems. We provide you the pseudo code for LQR-RRT\*.

``` python
def LQR_RRT_star(map_1, x_initial, n_tree, max_dist):
    
    tree = initiallize_tree(x_initial)
    
    for i in range (0, n_tree):
        x_rand = Sample(map_1)
        x_nearest = LQR_Nearest(tree, x_rand)
        [x_new, traj_new] = LQR_Steer(x_nearest, x_rand)
        x_near_set = LQR_Near(tree, x_new)
        [x_min, traj_min] = Choose_parent(x_near_set, x_new)
        
        if Collision_free(traj_min):
            tree = add_to_tree(x_new, traj_min)
            tree = Rewire(tree, x_near_set, x_new)
            
    return tree

def Choose_parent(x_near_set, x_new):
    
    min_cost = inf
    x_min = nil
    traj_min = nil
    
    for x_near in x_near_set:
        [x, traj] = LQR_Steer(x_near, x_new)
        if Cost(x_near) + Cost(traj) < min_cost:
            min_cost = Cost(x_near) + Cost(traj)
            x_min = x_near
            traj_min = traj
    
    return x_min, traj_min

def Rewire(tree, x_near_set, x_new):
    for x_near in x_near_set:
        [x, traj] = LQR_Steer(x_new, x_near)
        if Cost(x_new) + Cost(traj) < Cost(x_near):
            if Collision_free(traj):
                x_parent = Parent(x_near)
                tree = Remove_traj(x_parent, x_near)
                tree = add_traj(x_new, x_near)
    
    return tree
```

Note that LQR-RRT\* has the similar structure as RRT\*. 

The major differences:
    1. LQR-RRT* uses LQR controller based steering function, 
    2. We save calculated LQR control policy in the tree structure, instead of specific trajectories between two nodes.
    3. We use cost calculated from LQR to find the nearest node and set of near nodes from a randomly sampled
    configuration.

Now you can do the Mini-Pset! Good luck! Have fun!